In [1]:
from neo4j import GraphDatabase
import json
import os
import random
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
def snake_to_camel(snake_str):
    parts = snake_str.split('_')
    
    if len(parts) <= 1:
        return snake_str
    camel_case_str = parts[0] + "".join(part.capitalize() for part in parts[1:])
    
    return camel_case_str

In [3]:
URI = os.getenv('NEO4J_CONNECTION_URI')
AUTH = (os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))

In [17]:
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            result = session.run("RETURN 'That is right.' AS message")
            record = result.single()
            if record:
                print(record["message"])
            else:
                print("Query returned no result.")
except Exception as e:
    print(f"Connection or query failed: {e}")

That is right.


In [9]:
os.listdir('clean_data')

['papers',
 'funders',
 'topics',
 'authors',
 'domains.json',
 'fields.json',
 'institutions',
 'sources',
 'subfields.json']

In [11]:
with open('clean_data/domains.json', 'r', encoding='utf-8') as f:
    domains_data = json.load(f)
len(domains_data)

4

In [13]:
print(json.dumps(domains_data[0], indent=4))

{
    "id": "https://openalex.org/domains/3",
    "name": "Physical Sciences",
    "description": "branch of natural science that studies non-living systems",
    "fields": [
        "https://openalex.org/fields/15",
        "https://openalex.org/fields/16",
        "https://openalex.org/fields/17",
        "https://openalex.org/fields/19",
        "https://openalex.org/fields/21",
        "https://openalex.org/fields/22",
        "https://openalex.org/fields/23",
        "https://openalex.org/fields/25",
        "https://openalex.org/fields/26",
        "https://openalex.org/fields/31"
    ],
    "worksCount": 73237366,
    "citedByCount": 770120022
}


In [14]:
def import_domains(driver, domains_list):
    print(f"Starting import of {len(domains_list)} domains...")
    
    with driver.session() as session:

        total_imported = 0
        for domain in domains_list:
            props = domain.copy()
            
            if 'fields' in props:
                del props['fields']
            
            domain_id = props.pop('id', None)

            if not domain_id:
                print(f"Skipping domain, missing 'id': {domain}")
                continue

            try:
                session.run("""
                    MERGE (d:Domain {id: $id_param})
                    SET d += $props_param
                    """, id_param=domain_id, props_param=props)
                total_imported += 1
            except Exception as e:
                print(f"Failed to import domain {domain_id}: {e}")

    print(f"\nImport complete. {total_imported} domains merged/updated.")

In [15]:
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        import_domains(driver, domains_data)
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!
Starting import of 4 domains...

Import complete. 4 domains merged/updated.


In [29]:
query = """
MATCH (d:Domain)
RETURN d.name AS name, d.description AS description, d.worksCount AS worksCount
"""

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        records, summary, keys = driver.execute_query(query, database_="neo4j")
                
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [30]:
[rec.data() for rec in records]

[{'name': 'Physical Sciences',
  'description': 'branch of natural science that studies non-living systems',
  'worksCount': 73237366},
 {'name': 'Social Sciences',
  'description': 'branch of science focused on societies and the relationships among individuals within those societies',
  'worksCount': 64995390},
 {'name': 'Health Sciences',
  'description': 'branch of science focused on human health and disease prevention, diagnosis, treatment, and management',
  'worksCount': 43957129},
 {'name': 'Life Sciences',
  'description': 'branch of science that involve the scientific study of life – such as microorganisms, plants, and animals including human beings',
  'worksCount': 27536918}]

In [31]:
with open('clean_data/fields.json', 'r', encoding='utf-8') as f:
    fields_data = json.load(f)
len(fields_data)

26

In [32]:
print(json.dumps(fields_data[0], indent=4))

{
    "id": "https://openalex.org/fields/18",
    "name": "Decision Sciences",
    "description": "branch of applied probability theory",
    "nameAlternatives": [
        "decision theory",
        "theory of choice",
        "operations research"
    ],
    "domainId": "https://openalex.org/domains/2",
    "subfields": [
        "https://openalex.org/subfields/1800",
        "https://openalex.org/subfields/1802",
        "https://openalex.org/subfields/1803",
        "https://openalex.org/subfields/1804"
    ],
    "worksCount": 2096477,
    "citedByCount": 29367741
}


In [34]:
def import_fields(driver, fields_list):
    print(f"Starting import of {len(fields_list)} fields...")
    
    with driver.session() as session:

        total_imported = 0
        for field in fields_list:
            props = field.copy()
            
            if 'subfields' in props:
                del props['subfields']
            if 'domainId' in props:
                del props['domainId']
            
            field_id = props.pop('id', None)

            if not field_id:
                print(f"Skipping field, missing 'id': {field}")
                continue

            try:
                session.run("""
                    MERGE (d:Field {id: $id_param})
                    SET d += $props_param
                    """, id_param=field_id, props_param=props)
                total_imported += 1
            except Exception as e:
                print(f"Failed to import field {fields_list}: {e}")

    print(f"\nImport complete. {total_imported} field merged/updated.")

In [35]:
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        import_fields(driver, fields_data)
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!
Starting import of 26 fields...

Import complete. 26 field merged/updated.


In [39]:
query = """
MATCH (f:Field)
RETURN f.name AS name, f.description as description
"""

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        records, summary, keys = driver.execute_query(query, database_="neo4j")
                
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [44]:
records[4].data()

{'name': 'Engineering',
 'description': 'practice of using natural science, mathematics, and the engineering design process to solve technical problems'}

In [4]:
with open('clean_data/subfields.json', 'r', encoding='utf-8') as f:
    subfields_data = json.load(f)
len(subfields_data)

248

In [49]:
print(json.dumps(subfields_data[0], indent=4))

{
    "id": "https://openalex.org/subfields/2103",
    "name": "Fuel Technology",
    "description": "technology used in the extraction, processing and consumption of fuel resources",
    "nameAlternatives": [],
    "domainId": "https://openalex.org/domains/3",
    "fieldId": "https://openalex.org/fields/21",
    "topics": [
        "https://openalex.org/T14428"
    ],
    "worksCount": 46298,
    "citedByCount": 107545
}


In [3]:
def import_subfields(driver, subfields_list):
    print(f"Starting import of {len(subfields_list)} subfields...")
    
    with driver.session() as session:

        total_imported = 0
        for subfield in subfields_list:
            props = subfield.copy()
            

            if 'fieldId' in props:
                del props['fieldId']
            if 'domainId' in props:
                del props['domainId']
            if 'topics' in props:
                del props['topics']
            
            subfield_id = props.pop('id', None)

            if not subfield_id:
                print(f"Skipping subfield, missing 'id': {subfield}")
                continue

            try:
                session.run("""
                    MERGE (d:SubField {id: $id_param})
                    SET d += $props_param
                    """, id_param=subfield_id, props_param=props)
                total_imported += 1
            except Exception as e:
                print(f"Failed to import field {subfields_list}: {e}")

    print(f"\nImport complete. {total_imported} subfields merged/updated.")

In [5]:
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        import_subfields(driver, subfields_data)
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!
Starting import of 248 subfields...

Import complete. 248 subfields merged/updated.


In [16]:
query = """
MATCH (s:SubField)
RETURN s.name AS name, s.description AS description
"""

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        records, summary, keys = driver.execute_query(query, database_="neo4j")
                
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [20]:
records[50].data()

{'name': 'Information Systems',
 'description': 'academic study of systems with a specific reference to information and the complementary networks of hardware and software that people and organizations use to collect, filter, process, create and also distribute data'}

In [7]:
topics_data = []
topics_dir = os.path.join('clean_data', 'topics')

for fname in os.listdir(topics_dir):
    with open(os.path.join(topics_dir, fname), 'r', encoding='utf-8') as f:
        topics_data.extend(json.load(f))

random.shuffle(topics_data)
len(topics_data)

3655

In [8]:
print(json.dumps(topics_data[0], indent=4))

{
    "id": "https://openalex.org/T13722",
    "name": "Ergonomics and Human Factors",
    "description": "This cluster of papers explores the intersection of ergonomics and sustainability, focusing on human-compatible systems, work systems, sustainable development, environmental technology, and the application of agent-based simulation and digitalization. It delves into topics such as green ergonomics, polymer coatings, and the role of human factors in building design for sustainable work activities.",
    "keywords": [
        "Ergonomics",
        "Sustainability",
        "Human Factors",
        "Green Ergonomics",
        "Work Systems",
        "Sustainable Development",
        "Environmental Technology",
        "Agent-Based Simulation",
        "Digitalization",
        "Polymer Coatings"
    ],
    "domainId": "https://openalex.org/domains/3",
    "fieldId": "https://openalex.org/fields/22",
    "subfieldId": "https://openalex.org/subfields/2204",
    "worksCount": 12838,
  

In [ ]:
def import_topics_batch(driver, topics_list, ignore_key):
    print(f"Starting import of {len(topics_list)} topics...")
    
    props_list = []
    for topic in topics_list:
        props = topic.copy()

        for key in ignore_key:
            if key in props:
                del props[key]

        topic_id = props.pop('id', None)
        if topic_id:
            props_list.append({
                'id_param': topic_id,
                'props_param': props
            })


    with driver.session() as session:
        try:
            result = session.run("""
                UNWIND $props_list AS item
                MERGE (t:Topic {id: item.id_param})
                SET t += item.props_param
                RETURN count(t) AS total_merged
                """, props_list=props_list)
            
            summary = result.single()
            print(f"Batch import complete. {summary['total_merged']} topics merged/updated.")

        except Exception as e:
            print(f"Failed to import batch: {e}")

In [ ]:
N_topics = len(topics_data)
batch_size = 250

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        count = 1
        for i in range(0, N_topics, batch_size):
            selected_records = topics_data[i: i+batch_size]
            print(f'{count}:')
            import_topics_batch(driver, selected_records, ["domainId", "fieldId", "subfieldId"])
            count += 1
            print(2*'\n')
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [21]:
# query = """
# MATCH (t:Topic)
# RETURN t.id AS id
# """

query = """
MATCH (t:Topic)
RETURN t.name AS name, t.description AS description
"""

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        records, summary, keys = driver.execute_query(query, database_="neo4j")
                
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [22]:
# len(set([t.data().get('id') for t in records]))

In [24]:
records[16].data()

{'name': 'Economic Theory and Institutions',
 'description': 'This cluster of papers explores the study of economic institutions, behavior, and evolution, encompassing topics such as institutional economics, evolutionary economics, behavioral economics, market evolution, property rights, macroeconomics, social evolution, neoliberalism, and Darwinism in economics.'}

In [3]:
with open('clean_data/domains.json', 'r', encoding='utf-8') as f:
    domains_data = json.load(f)
len(domains_data)

4

In [4]:
domains_data[0]

{'id': 'https://openalex.org/domains/3',
 'name': 'Physical Sciences',
 'description': 'branch of natural science that studies non-living systems',
 'fields': ['https://openalex.org/fields/15',
  'https://openalex.org/fields/16',
  'https://openalex.org/fields/17',
  'https://openalex.org/fields/19',
  'https://openalex.org/fields/21',
  'https://openalex.org/fields/22',
  'https://openalex.org/fields/23',
  'https://openalex.org/fields/25',
  'https://openalex.org/fields/26',
  'https://openalex.org/fields/31'],
 'worksCount': 73237366,
 'citedByCount': 770120022}

In [5]:
domain_fields_relation = [
    {
        'domain_id_param': domain.get('id'),
        'field_ids_param': domain.get('fields'),
    } for domain in domains_data
]

In [7]:
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        with driver.session() as session:

            try:
                result = session.run("""
                    UNWIND $data_list AS item
                    
                    // 1. Find the existing Domain (fast due to constraint)
                    MATCH (d:Domain {id: item.domain_id_param})
                    
                    // 2. Unwind the list of field IDs for this domain
                    WITH d, item.field_ids_param AS field_ids
                    UNWIND field_ids AS field_id
                    
                    // 3. Find the existing Field (fast due to constraint)
                    MATCH (f:Field {id: field_id})
                    
                    // 4. Create the relationship (MERGE avoids duplicates)
                    MERGE (f)-[r:PART_OF]->(d)
                    
                    RETURN count(r) AS relationships_created
                    """, data_list=domain_fields_relation)
                
                summary = result.single()
                print(f"\nBatch import complete.")
                print(f"  - {summary['relationships_created']} relationships created/merged.")

            except Exception as e:
                # This will fail if a :Domain or :Field node is not found
                print(f"Failed to import batch: {e}")
                print("NOTE: This query assumes all :Domain and :Field nodes already exist.")
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!

Batch import complete.
  - 26 relationships created/merged.


In [10]:
query = """
MATCH (f:Field)-[:PART_OF]->(d:Domain {name: 'Physical Sciences'})
RETURN f.name AS field_name, f.description AS description
"""

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        records, summary, keys = driver.execute_query(query, database_="neo4j")
                
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [11]:
[rec.data() for rec in records]

[{'field_name': 'Physics and Astronomy',
  'description': 'study of matter and its motion, along with related concepts such as energy and force'},
 {'field_name': 'Mathematics',
  'description': 'area of knowledge that includes the topics of numbers, formulas and related structures, shapes and the spaces in which they are contained, and quantities and their changes'},
 {'field_name': 'Materials Science',
  'description': 'research, discovery and design of physical materials (especially solids)'},
 {'field_name': 'Environmental Science',
  'description': 'interdisciplinary field that studies human interaction with the environment'},
 {'field_name': 'Engineering',
  'description': 'practice of using natural science, mathematics, and the engineering design process to solve technical problems'},
 {'field_name': 'Energy',
  'description': 'study of energy in physical systems, its sources, technology, management, and impact on sustainability'},
 {'field_name': 'Earth and Planetary Sciences',

In [3]:
with open('clean_data/fields.json', 'r', encoding='utf-8') as f:
    fields_data = json.load(f)
len(fields_data)

26

In [4]:
fields_data[16]

{'id': 'https://openalex.org/fields/19',
 'name': 'Earth and Planetary Sciences',
 'description': 'all fields of natural science related to the planet Earth and other planets',
 'nameAlternatives': ['geosciences', 'planetary sciences'],
 'domainId': 'https://openalex.org/domains/3',
 'subfields': ['https://openalex.org/subfields/1902',
  'https://openalex.org/subfields/1904',
  'https://openalex.org/subfields/1906',
  'https://openalex.org/subfields/1907',
  'https://openalex.org/subfields/1908',
  'https://openalex.org/subfields/1910',
  'https://openalex.org/subfields/1911',
  'https://openalex.org/subfields/1912'],
 'worksCount': 4334720,
 'citedByCount': 65830936}

In [5]:
fields_subfields_relation = [
    {
        'field_id_param': field.get('id'),
        'subfield_ids_param': field.get('subfields'),
    } for field in fields_data
]
len(fields_subfields_relation)

26

In [8]:
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        with driver.session() as session:

            try:
                result = session.run("""
                    UNWIND $data_list AS item
                    
                    MATCH (f:Field {id: item.field_id_param})
                    
                    WITH f, item.subfield_ids_param AS subfield_ids
                    UNWIND subfield_ids AS subfield_id
                    
                    MATCH (s:SubField {id: subfield_id})
                    
                    MERGE (s)-[r:PART_OF]->(f)
                    
                    RETURN count(r) AS relationships_created
                    """, data_list=fields_subfields_relation)
                
                summary = result.single()
                print(f"\nBatch import complete.")
                print(f"  - {summary['relationships_created']} relationships created/merged.")

            except Exception as e:
                # This will fail if a :Domain or :Field node is not found
                print(f"Failed to import batch: {e}")
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!

Batch import complete.
  - 248 relationships created/merged.


In [10]:
query = """
MATCH (s:SubField)-[:PART_OF]->(f:Field {name: 'Engineering'})
RETURN s.name AS field_name, s.description AS description
"""

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        records, summary, keys = driver.execute_query(query, database_="neo4j")
                
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [11]:
print(json.dumps([rec.data() for rec in records], indent=4))

[
    {
        "field_name": "Safety, Risk, Reliability and Quality",
        "description": "state of being secure from harm, injury, danger, or other non-desirable outcomes"
    },
    {
        "field_name": "Mechanics of Materials",
        "description": "behavior of solid objects subject to stresses and strains"
    },
    {
        "field_name": "Ocean Engineering",
        "description": "engineering of devices within the ocean environment"
    },
    {
        "field_name": "Building and Construction",
        "description": "former West German trade union (1949\u20131995)"
    },
    {
        "field_name": "Mechanical Engineering",
        "description": "engineering discipline"
    },
    {
        "field_name": "Computational Mechanics",
        "description": "discipline concerned with the use of computational methods to study mechanics"
    },
    {
        "field_name": "General Engineering",
        "description": "applied science"
    },
    {
        "field_name": "

In [18]:
with open('clean_data/subfields.json', 'r', encoding='utf-8') as f:
    subfields_data = json.load(f)
len(subfields_data)

248

In [21]:
subfields_data[2].get('topics')

['https://openalex.org/T13509',
 'https://openalex.org/T12378',
 'https://openalex.org/T13366',
 'https://openalex.org/T10081',
 'https://openalex.org/T14183',
 'https://openalex.org/T14419',
 'https://openalex.org/T13972',
 'https://openalex.org/T14239',
 'https://openalex.org/T10019',
 'https://openalex.org/T12509',
 'https://openalex.org/T11508',
 'https://openalex.org/T12123',
 'https://openalex.org/T11653',
 'https://openalex.org/T13744',
 'https://openalex.org/T10517',
 'https://openalex.org/T11365',
 'https://openalex.org/T14180',
 'https://openalex.org/T14138',
 'https://openalex.org/T11903',
 'https://openalex.org/T12276',
 'https://openalex.org/T14030',
 'https://openalex.org/T12544']

In [22]:
subfields_topics_relation = [
    {
        'subfields_id_param': subfield.get('id'),
        'topics_ids_param': subfield.get('topics'),
    } for subfield in subfields_data
]
len(subfields_topics_relation)

248

In [24]:
subfields_topics_relation[2]

{'subfields_id_param': 'https://openalex.org/subfields/1402',
 'topics_ids_param': ['https://openalex.org/T13509',
  'https://openalex.org/T12378',
  'https://openalex.org/T13366',
  'https://openalex.org/T10081',
  'https://openalex.org/T14183',
  'https://openalex.org/T14419',
  'https://openalex.org/T13972',
  'https://openalex.org/T14239',
  'https://openalex.org/T10019',
  'https://openalex.org/T12509',
  'https://openalex.org/T11508',
  'https://openalex.org/T12123',
  'https://openalex.org/T11653',
  'https://openalex.org/T13744',
  'https://openalex.org/T10517',
  'https://openalex.org/T11365',
  'https://openalex.org/T14180',
  'https://openalex.org/T14138',
  'https://openalex.org/T11903',
  'https://openalex.org/T12276',
  'https://openalex.org/T14030',
  'https://openalex.org/T12544']}

In [25]:
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        with driver.session() as session:

            try:
                result = session.run("""
                    UNWIND $data_list AS item
                    
                    MATCH (s:SubField {id: item.subfields_id_param})
                    
                    WITH s, item.topics_ids_param AS topic_ids
                    UNWIND topic_ids AS topic_id
                    
                    MATCH (t:Topic {id: topic_id})
                    
                    MERGE (t)-[r:PART_OF]->(s)
                    
                    RETURN count(r) AS relationships_created
                    """, data_list=subfields_topics_relation)
                
                summary = result.single()
                print(f"\nBatch import complete.")
                print(f"  - {summary['relationships_created']} relationships created/merged.")

            except Exception as e:
                # This will fail if a :Domain or :Field node is not found
                print(f"Failed to import batch: {e}")
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!

Batch import complete.
  - 3655 relationships created/merged.


In [26]:
query = """
MATCH (t:Topic)-[:PART_OF]->(s:SubField {name: 'Control and Systems Engineering'})
RETURN t.name AS name, t.description AS description
"""

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        records, summary, keys = driver.execute_query(query, database_="neo4j")
                
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [27]:
print(json.dumps([rec.data() for rec in records], indent=4))

[
    {
        "name": "Magnetic Bearings and Levitation Dynamics",
        "description": "This cluster of papers focuses on the dynamics, control, and applications of magnetic levitation systems, including maglev trains, bearingless motors, active magnetic bearings, spacecraft attitude control using control moment gyros, vibration analysis of rotor systems, and fault diagnosis in rotating machinery."
    },
    {
        "name": "Extremum Seeking Control Systems",
        "description": "This cluster of papers focuses on extremum seeking control, a method for optimizing the performance of dynamic systems by iteratively adjusting control inputs to seek the extremum of a cost function. The papers cover various aspects such as stability analysis, adaptive control, stochastic optimization, and applications in fields like wind energy, bioreactors, and autonomous vehicles."
    },
    {
        "name": "Human Motion and Animation",
        "description": "This cluster of papers explores v

In [ ]:
sources_data = []
sources_dir = os.path.join('clean_data', 'sources')

for fname in os.listdir(sources_dir):
    with open(os.path.join(sources_dir, fname), 'r', encoding='utf-8') as f:
        sources_data.extend(json.load(f))

random.shuffle(sources_data)
len(sources_data)

835

In [31]:
sources_data[0].keys()

dict_keys(['id', 'name', 'hostOrganizationId', 'hostOrganizationName', 'worksCount', 'citedByCount', 'twoYearMeanCitedness', 'hIndex', 'iTenIndex', 'isOpenAccess', 'isInDoaj', 'isIndexedInScopus', 'isCore', 'ArticleProcessingCharge', 'countryCode', 'societies', 'topics', 'counts_by_year'])

In [48]:
sources_data[2].get('name')

'IEEE Transactions on Neural Networks'

In [ ]:
print(json.dumps(sources_data[3], indent=4))

In [11]:
snake_to_camel('counts_by_year')

'countsByYear'

In [22]:
def import_sources_batch(driver, sources_list, ignore_key, to_dump_list):
    print(f"Starting import of {len(sources_list)} sources...")
    
    props_list = []
    for source in sources_list:
        props = source.copy()

        for key in ignore_key:
            if key in props:
                del props[key]

        for key in to_dump_list:
            props[snake_to_camel(key)] = json.dumps(props.get(key, []))
            del props[key]

        topic_id = props.pop('id', None)
        if topic_id:
            props_list.append({
                'id_param': topic_id,
                'props_param': props
            })


    with driver.session() as session:
        try:
            result = session.run("""
                UNWIND $props_list AS item
                MERGE (s:Source {id: item.id_param})
                SET s += item.props_param
                RETURN count(s) AS total_merged
                """, props_list=props_list)
            
            summary = result.single()
            print(f"Batch import complete. {summary['total_merged']} sources merged/updated.")

        except Exception as e:
            print(f"Failed to import batch: {e}")

In [30]:
N_sources = len(sources_data)
batch_size = 250

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        count = 1
        for i in range(0, N_sources, batch_size):
            selected_records = sources_data[i: i+batch_size]
            print(f'{count}:')
            import_sources_batch(driver, selected_records, ["topics"], ['counts_by_year', 'ArticleProcessingCharge', 'societies'])
            count += 1
            print(2*'\n')
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!
1:
Starting import of 250 sources...
Batch import complete. 250 sources merged/updated.



2:
Starting import of 250 sources...
Batch import complete. 250 sources merged/updated.



3:
Starting import of 250 sources...
Batch import complete. 250 sources merged/updated.



4:
Starting import of 85 sources...
Batch import complete. 85 sources merged/updated.





In [34]:
# query = """
# MATCH (s:Source)
# RETURN s.id AS id
# """

query = """
MATCH (s:Source)
RETURN s.name AS name
"""

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        records, summary, keys = driver.execute_query(query, database_="neo4j")
                
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [35]:
# len(set([s.data().get('id') for s in records]))

In [36]:
[rec.data() for rec in records][:2]

[{'name': 'National Tax Journal'},
 {'name': 'International Journal of Vehicle Autonomous Systems'}]

In [43]:
sources_topics_relation = [
    {
        'source_id_param': source.get('id'),
        'topics_param': [
            {
                'id': topic.get('id'),
                'count': topic.get('count'),
            } for topic in source.get('topics', [])
        ]

    } for source in sources_data
]
len(sources_topics_relation)

835

In [45]:
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        with driver.session() as session:

            try:
                result = session.run("""
                    UNWIND $data_list AS item
                    
                    MATCH (s:Source {id: item.source_id_param})
                    
                    WITH s, item.topics_param AS topics_param
                    UNWIND topics_param AS topic_param
                    
                    MATCH (t:Topic {id: topic_param.id})
                    
                    MERGE (s)-[r:RELATED_TO]->(t)
                    SET r.count = topic_param.count
                    
                    RETURN count(r) AS relationships_created
                    """, data_list=sources_topics_relation)
                
                summary = result.single()
                print(f"\nBatch import complete.")
                print(f"  - {summary['relationships_created']} relationships created/merged.")

            except Exception as e:
                # This will fail if a :Domain or :Field node is not found
                print(f"Failed to import batch: {e}")
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!

Batch import complete.
  - 20843 relationships created/merged.


In [6]:
funders_data = []
funders_dir = os.path.join('clean_data', 'funders')

for fname in os.listdir(funders_dir):
    with open(os.path.join(funders_dir, fname), 'r', encoding='utf-8') as f:
        funders_data.extend(json.load(f))

random.shuffle(funders_data)
len(funders_data)

605

In [7]:
funders_data[0].keys()

dict_keys(['id', 'name', 'ror', 'doi', 'countryCode', 'description', 'grantsCount', 'worksCount', 'citedByCount', 'twoYearMeanCitedness', 'hIndex', 'iTenIndex', 'countsByYear'])

In [ ]:
print(json.dumps(funders_data[3], indent=4))

In [9]:
def import_funders_batch(driver, funders_list, ignore_key, to_dump_list):
    print(f"Starting import of {len(funders_list)} funders...")
    
    props_list = []
    for funder in funders_list:
        props = funder.copy()

        for key in ignore_key:
            if key in props:
                del props[key]

        for key in to_dump_list:
            props[snake_to_camel(key)] = json.dumps(props.get(key, []))
            del props[key]

        funder_id = props.pop('id', None)
        if funder_id:
            props_list.append({
                'id_param': funder_id,
                'props_param': props
            })


    with driver.session() as session:
        try:
            result = session.run("""
                UNWIND $props_list AS item
                MERGE (f:Funder {id: item.id_param})
                SET f += item.props_param
                RETURN count(f) AS total_merged
                """, props_list=props_list)
            
            summary = result.single()
            print(f"Batch import complete. {summary['total_merged']} funders merged/updated.")

        except Exception as e:
            print(f"Failed to import batch: {e}")

In [13]:
N_funders = len(funders_data)
batch_size = 250

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        count = 1
        for i in range(0, N_funders, batch_size):
            selected_records = funders_data[i: i+batch_size]
            print(f'{count}:')
            import_funders_batch(driver, selected_records, [], ['countsByYear'])
            count += 1
            print(2*'\n')
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!
1:
Starting import of 250 funders...
Batch import complete. 250 funders merged/updated.



2:
Starting import of 250 funders...
Batch import complete. 250 funders merged/updated.



3:
Starting import of 105 funders...
Batch import complete. 105 funders merged/updated.





In [14]:
query = """
MATCH (f:Funder)
RETURN f.name AS name
"""

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        records, summary, keys = driver.execute_query(query, database_="neo4j")
                
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [15]:
[rec.data() for rec in records[:5]]

[{'name': 'Korea Creative Content Agency'},
 {'name': 'Universitat Politècnica de València'},
 {'name': 'Ohio State University'},
 {'name': 'Office of Naval Research'},
 {'name': 'Fundação para a Ciência e a Tecnologia'}]

In [4]:
institutions_data = []
institutions_dir = os.path.join('clean_data', 'institutions')

for fname in os.listdir(institutions_dir):
    with open(os.path.join(institutions_dir, fname), 'r', encoding='utf-8') as f:
        institutions_data.extend(json.load(f))

random.shuffle(institutions_data)
len(institutions_data)

5048

In [5]:
institutions_data[0].keys()

dict_keys(['id', 'ror', 'name', 'countryCode', 'type', 'internationalName', 'worksCount', 'citedByCount', 'twoYearMeanCitedness', 'hIndex', 'iTenIndex', 'geo', 'countsByYear', 'topics'])

In [ ]:
print(json.dumps(institutions_data[0], indent=4))

In [7]:
def import_institutions_batch(driver, institutions_list, ignore_key, to_dump_list):
    print(f"Starting import of {len(institutions_list)} institutions...")
    
    props_list = []
    for institution in institutions_list:
        props = institution.copy()

        geo_data = props.pop('geo', None)

        for key in ignore_key:
            if key in props:
                del props[key]

        for key in to_dump_list:
            if key in props:
                camel_key = snake_to_camel(key)
                props[camel_key] = json.dumps(props.get(key, []))
                if key != camel_key:
                    del props[key]

        institution_id = props.pop('id', None)
        
        if institution_id:
            props_list.append({
                'id_param': institution_id,
                'props_param': props,       
                'geo_param': geo_data       
            })

    with driver.session() as session:
        try:
            result = session.run("""
                UNWIND $props_list AS item
                
                // 1. Merge the Institution node
                MERGE (i:Institution {id: item.id_param})
                SET i += item.props_param

                // 2. Conditionally merge the Geo node and the relationship
                // This FOREACH acts as a "IF item.geo_param IS NOT NULL THEN..."
                FOREACH (ignored IN CASE WHEN item.geo_param IS NOT NULL THEN [1] ELSE [] END |
                    // 2a. Merge the Geo node on its properties
                    MERGE (g:Geo {
                        city: item.geo_param.city, 
                        country: item.geo_param.country, 
                        latitude: item.geo_param.latitude, 
                        longitude: item.geo_param.longitude
                    })
                    // 2b. Merge the relationship
                    MERGE (i)-[:LOCATED_IN]->(g)
                )
                
                RETURN count(i) AS total_merged
                """, props_list=props_list)
            
            summary = result.single()
            print(f"Batch import complete. {summary['total_merged']} institutions merged/updated.")

        except Exception as e:
            print(f"Failed to import institution batch: {e}")

In [ ]:
N_institutions = len(institutions_data)
batch_size = 500

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        count = 1
        for i in range(0, N_institutions, batch_size):
            selected_records = institutions_data[i: i+batch_size]
            print(f'{count}:')
            import_institutions_batch(driver, selected_records, ['topics'], ['countsByYear'])
            count += 1
            print(2*'\n')
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [20]:
query = """
MATCH (g:Geo)
RETURN g.country AS country
"""


try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        records, summary, keys = driver.execute_query(query, database_="neo4j")
                
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [21]:
len(set([s.data().get('country') for s in records]))

106

In [22]:
[s.data().get('country') for s in records[:10]]

['United States',
 'United Kingdom',
 'South Africa',
 'Japan',
 'China',
 'India',
 'China',
 'Australia',
 'United States',
 'Switzerland']

In [23]:
institutions_topics_relation = [
    {
        'institution_id_param': institution.get('id'),
        'topics_param': [
            {
                'id': topic.get('id'),
                'count': topic.get('count'),
            } for topic in institution.get('topics', [])
        ]

    } for institution in institutions_data
]
len(institutions_topics_relation)

5048

In [24]:
institutions_topics_relation[0]

{'institution_id_param': 'https://openalex.org/I4210144354',
 'topics_param': [{'id': 'https://openalex.org/T10099', 'count': 42},
  {'id': 'https://openalex.org/T10022', 'count': 41},
  {'id': 'https://openalex.org/T11429', 'count': 21},
  {'id': 'https://openalex.org/T10655', 'count': 21},
  {'id': 'https://openalex.org/T10472', 'count': 19},
  {'id': 'https://openalex.org/T10262', 'count': 13},
  {'id': 'https://openalex.org/T10299', 'count': 11},
  {'id': 'https://openalex.org/T14117', 'count': 8},
  {'id': 'https://openalex.org/T12529', 'count': 8},
  {'id': 'https://openalex.org/T10326', 'count': 6},
  {'id': 'https://openalex.org/T10069', 'count': 6},
  {'id': 'https://openalex.org/T10020', 'count': 6},
  {'id': 'https://openalex.org/T10187', 'count': 6},
  {'id': 'https://openalex.org/T10125', 'count': 5},
  {'id': 'https://openalex.org/T11192', 'count': 5},
  {'id': 'https://openalex.org/T10622', 'count': 5},
  {'id': 'https://openalex.org/T11325', 'count': 5},
  {'id': 'https

In [26]:
N = len(institutions_topics_relation)
batch_size = 500

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        for i in range(0, N, batch_size):
            selected_data = institutions_topics_relation[i:i+batch_size]

            with driver.session() as session:

                try:
                    result = session.run("""
                        UNWIND $data_list AS item
                        
                        MATCH (i:Institution {id: item.institution_id_param})
                        
                        WITH i, item.topics_param AS topics_param
                        UNWIND topics_param AS topic_param
                        
                        MATCH (t:Topic {id: topic_param.id})
                        
                        MERGE (i)-[r:RELATED_TO]->(t)
                        SET r.count = topic_param.count
                        
                        RETURN count(r) AS relationships_created
                        """, data_list=selected_data)
                    
                    summary = result.single()
                    print(f"\nBatch import complete.")
                    print(f"  - {summary['relationships_created']} relationships created/merged.")
                    print()

                except Exception as e:
                    # This will fail if a :Domain or :Field node is not found
                    print(f"Failed to import batch: {e}")
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!

Batch import complete.
  - 12305 relationships created/merged.


Batch import complete.
  - 12349 relationships created/merged.


Batch import complete.
  - 12267 relationships created/merged.


Batch import complete.
  - 12363 relationships created/merged.


Batch import complete.
  - 12353 relationships created/merged.


Batch import complete.
  - 12352 relationships created/merged.


Batch import complete.
  - 12361 relationships created/merged.


Batch import complete.
  - 12305 relationships created/merged.


Batch import complete.
  - 12335 relationships created/merged.


Batch import complete.
  - 12335 relationships created/merged.


Batch import complete.
  - 1179 relationships created/merged.



In [4]:
authors_data = []
authors_dir = os.path.join('clean_data', 'authors')

for fname in os.listdir(authors_dir):
    with open(os.path.join(authors_dir, fname), 'r', encoding='utf-8') as f:
        authors_data.extend(json.load(f))

random.shuffle(authors_data)
len(authors_data)

17901

In [5]:
authors_data[0].keys()

dict_keys(['id', 'orcid', 'name', 'nameAlternatives', 'worksCount', 'numberOfCitation', 'twoYearMeanCitedness', 'hIndex', 'iTenIndex', 'affiliations', 'lastKnownInstitutions', 'topics', 'worksApiUrl', 'counts_by_year'])

In [6]:
print(json.dumps(authors_data[0], indent=4))

{
    "id": "https://openalex.org/A5011193488",
    "orcid": "https://orcid.org/0000-0001-8225-858X",
    "name": "Hu Cao",
    "nameAlternatives": [
        "Huasong Cao",
        "Hong\u2010fei Cao",
        "Hu Cao",
        "H. T. Cao",
        "Hongkun Cao",
        "H. Cao"
    ],
    "worksCount": 103,
    "numberOfCitation": 5367,
    "twoYearMeanCitedness": 76.25,
    "hIndex": 28,
    "iTenIndex": 55,
    "affiliations": [
        {
            "institution": "https://openalex.org/I62916508",
            "years": [
                2025,
                2024,
                2023,
                2022,
                2021,
                2020
            ]
        },
        {
            "institution": "https://openalex.org/I90610280",
            "years": [
                2025
            ]
        },
        {
            "institution": "https://openalex.org/I4210118629",
            "years": [
                2025
            ]
        },
        {
            "institut

In [7]:
def import_authors_batch(driver, authors_list, ignore_key, to_dump_list):
    print(f"Starting import of {len(authors_list)} authors...")
    
    props_list = []
    for author in authors_list:
        props = author.copy()

        for key in ignore_key:
            if key in props:
                del props[key]

        for key in to_dump_list:
            props[snake_to_camel(key)] = json.dumps(props.get(key, []))
            del props[key]

        author_id = props.pop('id', None)
        if author_id:
            props_list.append({
                'id_param': author_id,
                'props_param': props
            })


    with driver.session() as session:
        try:
            result = session.run("""
                UNWIND $props_list AS item
                MERGE (a:Author {id: item.id_param})
                SET a += item.props_param
                RETURN count(a) AS total_merged
                """, props_list=props_list)
            
            summary = result.single()
            print(f"Batch import complete. {summary['total_merged']} authors merged/updated.")

        except Exception as e:
            print(f"Failed to import batch: {e}")

In [ ]:
N_authors = len(authors_data)
batch_size = 1000

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        count = 1
        for i in range(0, N_authors, batch_size):
            selected_records = authors_data[i: i+batch_size]
            print(f'{count}:')
            import_authors_batch(driver, selected_records, ['affiliations', 'lastKnownInstitutions', 'topics'], ['counts_by_year'])
            count += 1
            print(2*'\n')
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [9]:
query = """
MATCH (a:Author)
RETURN a.id AS id
"""


try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        records, summary, keys = driver.execute_query(query, database_="neo4j")
                
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [11]:
len(set([s.data().get('id') for s in records]))

17900

In [12]:
authors_topics_relation = [
    {
        'author_id_param': author.get('id'),
        'topics_param': [
            {
                'id': topic.get('id'),
                'count': topic.get('count'),
            } for topic in author.get('topics', [])
        ]

    } for author in authors_data
]
len(authors_topics_relation)

17901

In [15]:
N = len(authors_topics_relation)
batch_size = 100

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        for i in range(0, N, batch_size):
            selected_data = authors_topics_relation[i:i+batch_size]

            with driver.session() as session:

                try:
                    result = session.run("""
                        UNWIND $data_list AS item
                        
                        MATCH (a:Author {id: item.author_id_param})
                        
                        WITH a, item.topics_param AS topics_param
                        UNWIND topics_param AS topic_param
                        
                        MATCH (t:Topic {id: topic_param.id})
                        
                        MERGE (a)-[r:RELATED_TO]->(t)
                        SET r.count = topic_param.count
                        
                        RETURN count(r) AS relationships_created
                        """, data_list=selected_data)
                    
                    summary = result.single()
                    # print(f"\nBatch import complete.")
                    print(f"  - {summary['relationships_created']} relationships created/merged.")
                    # print()

                except Exception as e:
                    # This will fail if a :Domain or :Field node is not found
                    print(f"Failed to import batch: {e}")
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!


  - 1460 relationships created/merged.
  - 1388 relationships created/merged.
  - 1390 relationships created/merged.
  - 1306 relationships created/merged.
  - 1359 relationships created/merged.
  - 1430 relationships created/merged.
  - 1275 relationships created/merged.
  - 1492 relationships created/merged.
  - 1572 relationships created/merged.
  - 1272 relationships created/merged.
  - 1371 relationships created/merged.
  - 1349 relationships created/merged.
  - 1446 relationships created/merged.
  - 1441 relationships created/merged.
  - 1202 relationships created/merged.
  - 1381 relationships created/merged.
  - 1452 relationships created/merged.
  - 1336 relationships created/merged.
  - 1496 relationships created/merged.
  - 1336 relationships created/merged.
  - 1348 relationships created/merged.
  - 1236 relationships created/merged.
  - 1407 relationships created/merged.
  - 1418 relationships created/merged.
  - 1437 relationships created/merged.
  - 1346 relationships cr

In [18]:
authors_lastKnownInstitutions_relation = [
    {
        'author_id': author.get('id'),
        'lastKnownInstitutions_ids': author.get('lastKnownInstitutions'),

    } for author in authors_data
]
len(authors_lastKnownInstitutions_relation)

17901

In [21]:
authors_lastKnownInstitutions_relation[2]

{'author_id': 'https://openalex.org/A5049077907',
 'lastKnownInstitutions_ids': ['https://openalex.org/I1294671590',
  'https://openalex.org/I4210159245']}

In [22]:
N = len(authors_lastKnownInstitutions_relation)
batch_size = 2000

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        for i in range(0, N, batch_size):
            selected_data = authors_lastKnownInstitutions_relation[i:i+batch_size]

            with driver.session() as session:

                try:
                    result = session.run("""
                        UNWIND $data_list AS item
                        
                        MATCH (a:Author {id: item.author_id})
                        
                        WITH a, item.lastKnownInstitutions_ids AS instituations_param
                        UNWIND instituations_param AS instituation_id
                        
                        MATCH (i:Institution {id: instituation_id})
                        
                        MERGE (a)-[r:LAST_KNOWN_AFFILIATION]->(i)
                        
                        RETURN count(r) AS relationships_created
                        """, data_list=selected_data)
                    
                    summary = result.single()
                    # print(f"\nBatch import complete.")
                    print(f"  - {summary['relationships_created']} relationships created/merged.")
                    # print()

                except Exception as e:
                    # This will fail if a :Domain or :Field node is not found
                    print(f"Failed to import batch: {e}")
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!
  - 2300 relationships created/merged.
  - 2323 relationships created/merged.
  - 2322 relationships created/merged.
  - 2368 relationships created/merged.
  - 2359 relationships created/merged.
  - 2292 relationships created/merged.
  - 2332 relationships created/merged.
  - 2337 relationships created/merged.
  - 2162 relationships created/merged.


In [24]:
print(json.dumps(authors_data[0].get('affiliations'), indent=4))

[
    {
        "institution": "https://openalex.org/I62916508",
        "years": [
            2025,
            2024,
            2023,
            2022,
            2021,
            2020
        ]
    },
    {
        "institution": "https://openalex.org/I90610280",
        "years": [
            2025
        ]
    },
    {
        "institution": "https://openalex.org/I4210118629",
        "years": [
            2025
        ]
    },
    {
        "institution": "https://openalex.org/I4210124652",
        "years": [
            2024
        ]
    },
    {
        "institution": "https://openalex.org/I4210132047",
        "years": [
            2024
        ]
    },
    {
        "institution": "https://openalex.org/I4210112065",
        "years": [
            2023
        ]
    },
    {
        "institution": "https://openalex.org/I6469544",
        "years": [
            2023
        ]
    },
    {
        "institution": "https://openalex.org/I204512498",
        "years": [
      

In [10]:
author_affiliations_relation = [
    {
        'author_id': author.get('id'),
        'affiliations_param': [
            {
                'id': affiliation.get('institution'),
                'years': affiliation.get('years'),
            } for affiliation in author.get('affiliations', [])
        ]

    } for author in authors_data
]
len(author_affiliations_relation)

17901

In [11]:
author_affiliations_relation[0]

{'author_id': 'https://openalex.org/A5103134187',
 'affiliations_param': [{'id': 'https://openalex.org/I4210164787',
   'years': [2025]},
  {'id': 'https://openalex.org/I194450716',
   'years': [2024, 2023, 2022, 2021, 2018, 2016, 2015, 2013, 2012, 2011]},
  {'id': 'https://openalex.org/I4392738231',
   'years': [2024, 2023, 2022, 2018, 2013]},
  {'id': 'https://openalex.org/I4210122543',
   'years': [2024, 2023, 2022, 2018, 2017, 2015, 2012, 2010]},
  {'id': 'https://openalex.org/I90610280',
   'years': [2023, 2022, 2020, 2019, 2018, 2010]},
  {'id': 'https://openalex.org/I181361081', 'years': [2023]},
  {'id': 'https://openalex.org/I4210127074', 'years': [2022]},
  {'id': 'https://openalex.org/I4210088511', 'years': [2022]},
  {'id': 'https://openalex.org/I198357462', 'years': [2022]},
  {'id': 'https://openalex.org/I98227222', 'years': [2022]}]}

In [8]:
N

17901

In [12]:
N = len(author_affiliations_relation)
batch_size = 100

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        for i in range(0, N, batch_size):
            selected_data = author_affiliations_relation[i:i+batch_size]

            with driver.session() as session:

                try:
                    result = session.run("""
                        UNWIND $data_list AS item
                        
                        MATCH (a:Author {id: item.author_id})
                        
                        WITH a, item.affiliations_param AS affiliations_param
                        UNWIND affiliations_param AS affiliation_param
                        
                        MATCH (i:Institution {id: affiliation_param.id})
                        
                        MERGE (a)-[r:AFFILIATED_WITH]->(i)
                        SET r.years = affiliation_param.years
                        
                        RETURN count(r) AS relationships_created
                        """, data_list=selected_data)
                    
                    summary = result.single()
                    # print(f"\nBatch import complete.")
                    print(f"  - {summary['relationships_created']} relationships created/merged.")
                    # print()

                except Exception as e:
                    # This will fail if a :Domain or :Field node is not found
                    print(f"Failed to import batch: {e}")
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!


  - 548 relationships created/merged.
  - 516 relationships created/merged.
  - 560 relationships created/merged.
  - 516 relationships created/merged.
  - 542 relationships created/merged.
  - 484 relationships created/merged.
  - 536 relationships created/merged.
  - 572 relationships created/merged.
  - 527 relationships created/merged.
  - 541 relationships created/merged.
  - 581 relationships created/merged.
  - 533 relationships created/merged.
  - 552 relationships created/merged.
  - 585 relationships created/merged.
  - 515 relationships created/merged.
  - 580 relationships created/merged.
  - 577 relationships created/merged.
  - 562 relationships created/merged.
  - 528 relationships created/merged.
  - 572 relationships created/merged.
  - 515 relationships created/merged.
  - 574 relationships created/merged.
  - 578 relationships created/merged.
  - 537 relationships created/merged.
  - 577 relationships created/merged.
  - 545 relationships created/merged.
  - 531 rela

In [4]:
papers_data = []
papers_dir = os.path.join('clean_data', 'papers')

for fname in os.listdir(papers_dir):
    with open(os.path.join(papers_dir, fname), 'r', encoding='utf-8') as f:
        papers_data.extend(json.load(f))

random.shuffle(papers_data)
len(papers_data)

6671

In [5]:
papers_data[0].keys()

dict_keys(['paper', 'authorship', 'source', 'referencedWorks', 'relatedWorks', 'grants', 'concepts', 'keywords', 'topics'])

In [18]:
print(json.dumps(papers_data[0], indent=4))

{
    "paper": {
        "id": "https://openalex.org/W3175984392",
        "doi": "https://doi.org/10.1016/j.cose.2021.102269",
        "title": "A survey on security attacks and defense techniques for connected and autonomous vehicles",
        "publicationYear": 2021,
        "language": "en",
        "type": "article",
        "crossrefType": "journal-article",
        "numberOfCiteation": 145,
        "FieldWeightedCitationImpact": 9.612,
        "countsByYear": [
            {
                "year": 2025,
                "cited_by_count": 24
            },
            {
                "year": 2024,
                "cited_by_count": 37
            },
            {
                "year": 2023,
                "cited_by_count": 46
            },
            {
                "year": 2022,
                "cited_by_count": 30
            },
            {
                "year": 2021,
                "cited_by_count": 7
            },
            {
                "year": 2020,
    

In [9]:
papers_data[0].get('keywords')

['Interconnectivity']

In [12]:
papers_info = [{**p.get('paper',{}), 'keywords': p.get('keywords',[])} for p in papers_data]

In [17]:
print(json.dumps(papers_info[2], indent=4))

{
    "id": "https://openalex.org/W2783651538",
    "doi": "https://doi.org/10.3390/rs10010072",
    "title": "Double Weight-Based SAR and Infrared Sensor Fusion for Automatic Ground Target Recognition with Deep Learning",
    "publicationYear": 2018,
    "language": "en",
    "type": "article",
    "crossrefType": "journal-article",
    "numberOfCiteation": 33,
    "FieldWeightedCitationImpact": 2.778,
    "countsByYear": [
        {
            "year": 2025,
            "cited_by_count": 5
        },
        {
            "year": 2024,
            "cited_by_count": 6
        },
        {
            "year": 2023,
            "cited_by_count": 5
        },
        {
            "year": 2022,
            "cited_by_count": 2
        },
        {
            "year": 2021,
            "cited_by_count": 4
        },
        {
            "year": 2020,
            "cited_by_count": 6
        },
        {
            "year": 2019,
            "cited_by_count": 3
        },
        {
        

In [15]:
def import_papers_batch(driver, papers_list, ignore_key, to_dump_list):
    print(f"Starting import of {len(papers_list)} papers...")
    
    props_list = []
    for paper in papers_list:
        props = paper.copy()

        for key in ignore_key:
            if key in props:
                del props[key]

        for key in to_dump_list:
            props[snake_to_camel(key)] = json.dumps(props.get(key, []))
            del props[key]

        paper_id = props.pop('id', None)
        if paper_id:
            props_list.append({
                'id_param': paper_id,
                'props_param': props
            })


    with driver.session() as session:
        try:
            result = session.run("""
                UNWIND $props_list AS item
                MERGE (p:Paper {id: item.id_param})
                SET p += item.props_param
                RETURN count(p) AS total_merged
                """, props_list=props_list)
            
            summary = result.single()
            print(f"Batch import complete. {summary['total_merged']} papers merged/updated.")

        except Exception as e:
            print(f"Failed to import batch: {e}")

In [19]:
N_papers = len(papers_info)
batch_size = 1000

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        count = 1
        for i in range(0, N_papers, batch_size):
            selected_records = papers_info[i: i+batch_size]
            print(f'{count}:')
            import_papers_batch(driver, selected_records, [], ['countsByYear'])
            count += 1
            print(2*'\n')
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!
1:
Starting import of 1000 papers...
Batch import complete. 1000 papers merged/updated.



2:
Starting import of 1000 papers...
Batch import complete. 1000 papers merged/updated.



3:
Starting import of 1000 papers...
Batch import complete. 1000 papers merged/updated.



4:
Starting import of 1000 papers...
Batch import complete. 1000 papers merged/updated.



5:
Starting import of 1000 papers...
Batch import complete. 1000 papers merged/updated.



6:
Starting import of 1000 papers...
Batch import complete. 1000 papers merged/updated.



7:
Starting import of 671 papers...
Batch import complete. 671 papers merged/updated.





In [20]:
papers_data[0].keys()

dict_keys(['paper', 'authorship', 'source', 'referencedWorks', 'relatedWorks', 'grants', 'concepts', 'keywords', 'topics'])

### paper - source relation

In [31]:
papers_data[2].get('source')

{'id': 'https://openalex.org/S43295729',
 'hostOrganizationId': 'https://openalex.org/P4310310987',
 'volume': '10',
 'issue': '1'}

In [34]:
papers_sources_relation = [
    {
        'paper_id': paper.get('paper', {}).get('id'),
        'source_param': {
                'id': paper.get('source', {}).get('id'),
                'params': {
                    'volume': paper.get('source', {}).get('volume'),
                    'issue': paper.get('source', {}).get('issue')
                }
        } 
        

    } for paper in papers_data
]
len(papers_sources_relation)

6671

In [35]:
papers_sources_relation[2]

{'paper_id': 'https://openalex.org/W2783651538',
 'source_param': {'id': 'https://openalex.org/S43295729',
  'params': {'volume': '10', 'issue': '1'}}}

In [36]:
N = len(papers_sources_relation)
batch_size = 500

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("✅ Connection successful!")
        
        for i in range(0, N, batch_size):
            selected_data = papers_sources_relation[i:i+batch_size]

            with driver.session() as session:

                try:
                    result = session.run("""
                        UNWIND $data_list AS item
                        
                        MATCH (p:Paper {id: item.paper_id})
                        
                        WITH p, item.source_param AS source_param
                        
                        MATCH (s:Source {id: source_param.id})
                        
                        MERGE (p)-[r:PUBLISHED_IN]->(s)
                        SET r += source_param.params
                        
                        RETURN count(r) AS relationships_created
                        """, data_list=selected_data)
                    
                    summary = result.single()
                    # print(f"\nBatch import complete.")
                    print(f"  - {summary['relationships_created']} relationships created/merged.")
                    # print()

                except Exception as e:
                    # This will fail if a :Domain or :Field node is not found
                    print(f"Failed to import batch: {e}")
        
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Connection successful!
  - 400 relationships created/merged.
  - 404 relationships created/merged.
  - 410 relationships created/merged.
  - 406 relationships created/merged.
  - 420 relationships created/merged.
  - 425 relationships created/merged.
  - 408 relationships created/merged.
  - 407 relationships created/merged.
  - 413 relationships created/merged.
  - 399 relationships created/merged.
  - 394 relationships created/merged.
  - 390 relationships created/merged.
  - 411 relationships created/merged.
  - 138 relationships created/merged.


### paper - grant relation

In [26]:
papers_data[1356].get('grants')

[{'funder': 'https://openalex.org/F4320321001', 'awardId': 'T2350710232'},
 {'funder': 'https://openalex.org/F4320321001', 'awardId': '62466025'}]

### paper - topics relation

In [27]:
papers_data[1356].get('topics')

['https://openalex.org/T10586',
 'https://openalex.org/T11615',
 'https://openalex.org/T10879']

### paper - authorship relation

In [28]:
papers_data[1356].get('authorship')

[{'authorPosition': 'first',
  'author': {'id': 'https://openalex.org/A5042245303',
   'institutions': [],
   'isCorresponding': False,
   'affiliations': []}},
 {'authorPosition': 'middle',
  'author': {'id': 'https://openalex.org/A5077930860',
   'institutions': [],
   'isCorresponding': False,
   'affiliations': []}},
 {'authorPosition': 'last',
  'author': {'id': 'https://openalex.org/A5000202665',
   'institutions': [],
   'isCorresponding': False,
   'affiliations': []}}]

### paper - cite relation

In [29]:
papers_data[1356].get('referencedWorks')

['https://openalex.org/W1522301498',
 'https://openalex.org/W1971086298',
 'https://openalex.org/W1971458750',
 'https://openalex.org/W1992157405',
 'https://openalex.org/W2000194319',
 'https://openalex.org/W2296073425',
 'https://openalex.org/W2306644740',
 'https://openalex.org/W2611243847',
 'https://openalex.org/W2757731190',
 'https://openalex.org/W2806844078',
 'https://openalex.org/W2912063360',
 'https://openalex.org/W2926153681',
 'https://openalex.org/W2953248129',
 'https://openalex.org/W2964248288',
 'https://openalex.org/W2967969632',
 'https://openalex.org/W2968385028',
 'https://openalex.org/W3004162282']